In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("VSCodeTest") \
    .master("local[*]") \
    .getOrCreate()

print("✅ Spark session running in VS Code!")
print("Spark version:", spark.version)

✅ Spark session running in VS Code!
Spark version: 4.0.0


In [ ]:
from openai import OpenAI
from IPython.display import Markdown, display
import unicodedata
import re

# Connect to your local Ollama server
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"  # Dummy key required by SDK
)

def ask_local_gpt(prompt: str, model="gpt-oss:20b", system_message=None, render_markdown=True):
    messages = []
    if system_message:
        messages.append({"role": "system", "content": system_message})
    messages.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    raw_output = response.choices[0].message.content

    # Normalize spacing but preserve formatting
    clean_output = unicodedata.normalize("NFKC", raw_output).replace("\u202f", " ")

    if render_markdown:
        display(Markdown(clean_output))
    return clean_output


In [38]:
import time
import unicodedata
import re
from typing import Optional
from openai import OpenAI
from IPython.display import Markdown, display

# Connect to your local Ollama server
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"  # Dummy key required by SDK
)

In [39]:
conversation_log = []
chat_memory = []

def ask_local_gpt(
    prompt: str,
    model: str = "gpt-oss:20b",
    system_message: Optional[str] = None,
    render_markdown: bool = True,
    verbose: bool = False,
    return_raw: bool = False,
    reset_chat: bool = False,
    show_history: bool = False,
    stream_response: bool = True,
    reasoning_mode: bool = False
) -> Optional[str]:
    """
    Query a local GPT model via Ollama, with support for step-by-step reasoning (Scratchpad).

    Args:
        prompt (str): The user’s input/question.
        model (str): Model name (default 'gpt-oss:20b').
        system_message (str, optional): Optional system prompt.
        render_markdown (bool): If True, pretty-print answer as markdown in notebook.
        verbose (bool): If True, print extra info.
        return_raw (bool): If True, return the full unprocessed model response.
        reset_chat (bool): If True, reset the conversation history.
        show_history (bool): If True, print the message history after each call.
        stream_response (bool): If True, stream the response token by token.
        reasoning_mode (bool): If True, injects scratchpad ("thinking out loud") prompt.

    Returns:
        str or None: Clean final answer or full output, depending on flags.
    """
    global chat_memory, conversation_log

    try:
        if reset_chat:
            chat_memory = []

        # System prompt for reasoning mode
        default_reasoning_prompt = (
            "You are a helpful assistant that always reasons step by step before giving an answer. "
            "First, think through the problem, then provide a clear final answer."
        )

        # Add a system message only if not present
        if not any(m.get("role") == "system" for m in chat_memory):
            chat_memory.insert(0, {
                "role": "system",
                "content": system_message or (default_reasoning_prompt if reasoning_mode else "")
            })

        # Prefix the prompt with reasoning instructions if enabled
        if reasoning_mode:
            full_prompt = (
                f"### Scratchpad:\n"
                f"The user asked: \"{prompt.strip()}\"\n"
                f"Think step-by-step and reason before answering.\n\n"
                f"### Final Answer:\n"
            )
        else:
            full_prompt = prompt

        chat_memory.append({"role": "user", "content": full_prompt})
        start = time.time()

        # Get the response (stream or not)
        if stream_response:
            print("🤔 Thinking...\n")
            stream = client.chat.completions.create(
                model=model,
                messages=chat_memory,
                stream=True
            )
            tokens = []
            for chunk in stream:
                delta = chunk.choices[0].delta.content or ""
                print(delta, end="", flush=True)
                tokens.append(delta)
            print()
            assistant_reply = "".join(tokens)
        else:
            response = client.chat.completions.create(
                model=model,
                messages=chat_memory
            )
            assistant_reply = response.choices[0].message.content

        chat_memory.append({"role": "assistant", "content": assistant_reply})

        # ---- Parse the thoughts/scratchpad, final answer, and raw ----
        # Default values
        thoughts = ""
        final_answer = assistant_reply.strip()

        # Extract "Scratchpad" (step-by-step reasoning) and "Final Answer"
        scratchpad_match = re.search(
            r"### Scratchpad:\s*(.*?)### Final Answer:",
            assistant_reply,
            re.DOTALL | re.IGNORECASE
        )
        final_answer_match = re.search(
            r"### Final Answer:\s*([\s\S]*?)(?:$|\n#|\n\n)",
            assistant_reply,
            re.DOTALL | re.IGNORECASE
        )

        if scratchpad_match:
            # Remove markdown, extra whitespace, etc. from scratchpad
            thoughts = scratchpad_match.group(1).strip()
        if final_answer_match:
            # Strip any trailing markdown/whitespace for the cleanest final answer
            final_answer = final_answer_match.group(1).strip()

        # Save everything to log for later analysis/export
        conversation_log.append({
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
            "prompt": prompt,
            "thoughts": thoughts,
            "answer": final_answer,
            "raw": assistant_reply
        })

        # Choose output behavior based on flags
        if return_raw:
            return assistant_reply
        elif render_markdown:
            display(Markdown(final_answer))
        elif verbose:
            print("\n🧼 Clean answer:\n", final_answer)
        else:
            return final_answer

        if verbose:
            print(f"\n✅ Response time: {round(time.time() - start, 2)}s")

        if show_history:
            print("\n📜 Message History:")
            for msg in chat_memory:
                print(f"{msg['role'].upper()}: {msg['content']}\n")

        return final_answer if verbose else None

    except Exception as e:
        print("❌ Error in ask_local_gpt:", str(e))
        return "Error occurred."


In [40]:
ask_local_gpt("What's 1 + 1?", stream_response=True, reasoning_mode=True);

🤔 Thinking...

To solve the problem, we perform a simple addition:

1. Start with the first number: **1**.
2. Add the second number: **1**.
3. Perform the addition: \(1 + 1 = 2\).

So, **1 + 1 equals 2**.


To solve the problem, we perform a simple addition:

1. Start with the first number: **1**.
2. Add the second number: **1**.
3. Perform the addition: \(1 + 1 = 2\).

So, **1 + 1 equals 2**.

In [41]:
import pandas as pd
df_log = pd.DataFrame(conversation_log)
df_log.to_csv("chat_log.csv", index=False)

In [42]:
df_log

,timestamp,prompt,thoughts,answer,raw
0,2025-08-06 22:01:54,What's 1 + 1?,,"To solve the problem, we perform a simple addi...","To solve the problem, we perform a simple addi..."


In [2]:
spark.stop()